In [57]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pd.set_option('display.max_columns', None)
train = pd.read_csv("/kaggle/input/big-data-analytics-certification/t2-1-train.csv")
# train.head()
# train.info()
test = pd.read_csv("/kaggle/input/big-data-analytics-certification/t2-1-test.csv")
# test.head()
# test.info()
#objcols = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']

# train.describe()
# train.describe(include = "O")
# test.describe()
# test.describe(include = "O")

train.isnull().sum() #AnnualIncome 결측치
test.isnull().sum() #여기도 AnnualIncome 결측치

train_med = train['AnnualIncome'].median()
test_med = train['AnnualIncome'].median()
train['AnnualIncome'] = train['AnnualIncome'].fillna(train_med)
test['AnnualIncome'] = test['AnnualIncome'].fillna(test_med)

#train['Employment Type'].unique() #train에는 2개 형태뿐인데, test에 3개? -> 예측이 이상할 것 같은데? 칼럼 드롭하고 안하고 차이를 확인해보자
train = train.drop('Employment Type', axis = 1) #todo : 이상한 칼럼 하나 있어보이는데 드랍해보고 안해보고 차이 확인 예정
test = test.drop('Employment Type', axis = 1) 
train = pd.get_dummies(train)
#train.head()
test = pd.get_dummies(test)
#test.head()
#train['Employment Type_Casual employment'] = False #강제로 all false 때려서 칼럼 추가
#train.head()
train = train.drop('id', axis = 1)
test_id = test.pop('id')
target = train.pop('TravelInsurance')

# train.info()
# test.info()

#분할
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(train, target, test_size = 0.2, random_state = 0)
#x_tr.shape, x_val.shape, y_tr.shape, y_val.shape  #((1192, 10), (298, 10), (1192,), (298,))

#구매 확률 값 -> 랜포 분류, predict proba
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 0)
model.fit(x_tr, y_tr)
pred = model.predict_proba(x_val)
pred

from sklearn.metrics import roc_auc_score
ras = roc_auc_score(y_val, pred[:, 1])
#print(ras) 
#0.8081627680311891 - 고용형태 드랍했을 때 / 드랍하지 않으면 성능이 올라가긴 할텐데, 칼럼이 맞지 않는다. 강제로 추가하려고 해도 순서를 맞춰야 한다. 그래서 그냥 쿨하게 버림

pred = model.predict_proba(test)


submission = pd.DataFrame({'id' : test_id, 'TravelInsurance' : pred[:, 1] })
submission.to_csv("submission.csv", index = False)
pd.read_csv("submission.csv")


/kaggle/input/big-data-analytics-certification/t2-2-y_train.csv
/kaggle/input/big-data-analytics-certification/t1-data1.csv
/kaggle/input/big-data-analytics-certification/t2-2-X_train.csv
/kaggle/input/big-data-analytics-certification/t2-1-sample_submission.csv
/kaggle/input/big-data-analytics-certification/t1-data2.csv
/kaggle/input/big-data-analytics-certification/t2-1-test.csv
/kaggle/input/big-data-analytics-certification/t2-2-X_test.csv
/kaggle/input/big-data-analytics-certification/t2-1-train.csv


,id,TravelInsurance
0,0,0.239000
1,1,0.000000
2,2,0.109000
3,3,1.000000
4,4,0.092667
...,...,...
492,492,0.010000
493,493,0.950000
494,494,0.230000
495,495,1.000000
